In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os
import cv2

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D , MaxPooling2D, UpSampling2D,Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline 

2023-06-27 16:31:30.620656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
for dirname, _, filenames in os.walk('/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
path_zip='/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/denoising-dirty-documents/'
path='/scanpunjabi/working'

/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-68.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-54.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-97.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-83.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/1-84.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/1-90.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/1-91.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/1-85.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-82.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-96.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-55.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-69.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-57.tif
/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/a-80.tif
/Users

In [7]:
with zipfile.ZipFile(path_zip+'train.zip','r') as zip_ref:
    zip_ref.extractall(path)
with zipfile.ZipFile(path_zip+'test.zip','r') as zip_ref:
    zip_ref.extractall(path)
with zipfile.ZipFile(path_zip+'train_cleaned.zip','r') as zip_ref:
    zip_ref.extractall(path)
with zipfile.ZipFile(path_zip+'sampleSubmission.csv.zip','r') as zip_ref:
    zip_ref.extractall(path)
train_img=sorted(os.listdir(path+'/train'))
train_cleaned_img=sorted(os.listdir(path+'/train_cleaned'))
test_img=sorted(os.listdir(path+'/test'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rushangipatel/Documents/GitHub/Scanpunjabi/Punjabidata/denoising-dirty-documents/train.zip'

In [ ]:
def process_image(path):
    img = cv2.imread(path)
    img = np.asarray(img, dtype = "float32")
    img = cv2.resize(img, (540, 420))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255.0 
    img = np.reshape(img, (420, 540, 1))
    
    return img

In [ ]:
train = []
train_cleaned = []
test = []

for f in sorted(os.listdir(path+'train/')):
    train.append(process_image(path+ 'train/'+f))
    
for f in sorted(os.listdir(path+'train_cleaned/')):
    train_cleaned.append(process_image(path+ 'train_cleaned/'+f))
    
for f in sorted(os.listdir(path+'test/')):
    test.append(process_image(path+ 'test/'+f))

In [ ]:
plt.figure(figsize=(15,25))     #size of 15 inches by 25 inches.
for i in range(0,8,2):          #Loops 0 to 8 with a step of 2, which will generate 4 iterations.
    plt.subplot(4,2,i+1)        # 4 rows and 2 columns, sets the current subplot to the ith position in the grid (i+1 to account for zeroindexing).
    plt.xticks([])              #Removes the x-axis and y-axis tick labels.
    plt.yticks([])
    plt.imshow(train[i][:,:,0],cmap='gray')            #display image in grayscale.
    plt.title('Noise image: {}'.format(train_img[i]))
    
    plt.subplot(4,2,i+2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_cleaned[i][:,:,0],cmap='gray')
    plt.title('Denoised image: {}'.format(train_img[i]))
    
plt.show()
    

In [ ]:
X_train =  np.asarray(train)
Y_train =  np.asarray(train_cleaned)
X_test = np.asarray(test)
X_train,X_val, Y_train,Y_val = train_test_split(X_train,Y_train,test_size=0.15)

In [ ]:
def model() : 
    input_layer=Input (shape=(420,540,1))
    #encoding
    x= Conv2D(64,(3,3),activation='relu',padding='same')(input_layer)
    x= Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x= BatchNormalization()(x)
    
    x= MaxPooling2D((2,2),padding='same')(x)
    
    x= Dropout(0.5)(x)
    
    #decoding
    x= Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x= Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x= BatchNormalization()(x)
    x= UpSampling2D((2,2))(x)
    
    output_layer = Conv2D(1,(3,3),activation='sigmoid',padding='same')(x)
    model= Model(inputs=[input_layer],outputs=[output_layer])
    model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
    
    return model

model = model()
model.summary()

In [ ]:
callback= EarlyStopping(monitor='loss',patience=30)
history= model.fit(X_train,Y_train,validation_data= (X_val,Y_val),epochs=300,batch_size=12, verbose=1, callbacks=[callback])

In [ ]:
epoch_loss = history.history['loss']
epoch_val_loss = history.history['val_loss']
epoch_mae = history.history['mae']
epoch_val_mae =  history.history['val_mae']

plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
plt.plot(range(0,len(epoch_loss)),epoch_loss,'b-',linewidth=2, label='Train Loss')
plt.plot(range(0,len(epoch_val_loss)),epoch_val_loss,'r-',linewidth=2, label='Val Loss')
plt.title('Evolution of loss on train and validation datasets over epochs')
plt.legend(loc='best')

plt.subplot(1,2,2)
plt.plot(range(0,len(epoch_mae)),epoch_mae,'b-',linewidth=2,label='Train MAE')
plt.plot(range(0,len(epoch_val_mae)),epoch_val_mae,'r-',linewidth=2,label='VAL MAE')
plt.title('Evolution of MAE on train and validation datasets over epochs')
plt.legend(loc='best')

plt.show

In [ ]:
Y_test = model.predict(X_test, batch_size=16)

In [ ]:
plt.figure(figsize=(15,25))
for i in range (0,8,2):
    plt.subplot(4,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_test[i][:,:,0],cmap='gray')
    plt.title('Noisy image: {}'.format(test_img[i]))
    
    plt.subplot(4,2,i+2)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(Y_test[i][:,:,0],cmap='gray')
    plt.title('Denoised by autoencoder: {}'.format(test_img[i]))

In [ ]:
model_json =  model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)

#serialize weight to HDF5
model.save_weights("model.h5") #model/ae
print("Saved model to disk")